In [15]:
import torch
import torchvision
import torchvision.transforms as transforms

In [16]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

# Im guessing that the test images are the same as the trianing ones, from the way it downloads them
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [17]:
trainset[0]

(
 (0 ,.,.) = 
  -0.5373 -0.6627 -0.6078  ...   0.2392  0.1922  0.1608
  -0.8745 -1.0000 -0.8588  ...  -0.0353 -0.0667 -0.0431
  -0.8039 -0.8745 -0.6157  ...  -0.0745 -0.0588 -0.1451
            ...             ⋱             ...          
   0.6314  0.5765  0.5529  ...   0.2549 -0.5608 -0.5843
   0.4118  0.3569  0.4588  ...   0.4431 -0.2392 -0.3490
   0.3882  0.3176  0.4039  ...   0.6941  0.1843 -0.0353
 
 (1 ,.,.) = 
  -0.5137 -0.6392 -0.6235  ...   0.0353 -0.0196 -0.0275
  -0.8431 -1.0000 -0.9373  ...  -0.3098 -0.3490 -0.3176
  -0.8118 -0.9451 -0.7882  ...  -0.3412 -0.3412 -0.4275
            ...             ⋱             ...          
   0.3333  0.2000  0.2627  ...   0.0431 -0.7569 -0.7333
   0.0902 -0.0353  0.1294  ...   0.1608 -0.5137 -0.5843
   0.1294  0.0118  0.1137  ...   0.4431 -0.0745 -0.2784
 
 (2 ,.,.) = 
  -0.5059 -0.6471 -0.6627  ...  -0.1529 -0.2000 -0.1922
  -0.8431 -1.0000 -1.0000  ...  -0.5686 -0.6078 -0.5529
  -0.8353 -1.0000 -0.9373  ...  -0.6078 -0.6078 -0.6706
   

In [3]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

 bird   dog   dog  bird


Process Process-1:
Process Process-2:
Traceback (most recent call last):
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/riccardo/anaconda2/envs/py36/lib/p

In [4]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

inputs: 

(0 ,0 ,.,.) = 
 -0.0902 -0.1059 -0.1216  ...   0.6235  0.7412  0.7098
 -0.0353 -0.0588 -0.0667  ...   0.5216  0.7255  0.7020
  0.0588  0.0588  0.0824  ...   0.4275  0.7412  0.7490
           ...             ⋱             ...          
  0.6314  0.6314  0.6471  ...   0.6941  0.7098  0.7020
  0.6784  0.6392  0.6314  ...   0.6863  0.6784  0.6627
  0.6784  0.6235  0.6157  ...   0.6863  0.6314  0.6314

(0 ,1 ,.,.) = 
  0.1686  0.1529  0.1294  ...   0.6000  0.6784  0.6549
  0.1922  0.1686  0.1686  ...   0.5059  0.6784  0.6627
  0.2627  0.2549  0.2784  ...   0.4196  0.7020  0.7176
           ...             ⋱             ...          
  0.7882  0.7725  0.7882  ...   0.8431  0.8588  0.8510
  0.8275  0.7882  0.7804  ...   0.8353  0.8275  0.8118
  0.8275  0.7725  0.7647  ...   0.8353  0.7804  0.7725

(0 ,2 ,.,.) = 
 -0.0431 -0.0588 -0.0745  ...   0.3255  0.3725  0.3490
 -0.0196 -0.0431 -0.0510  ...   0.2314  0.3804  0.3647
  0.0353  0.0275  0.0588  ...   0.1608  0.4118  0.4353
        


(0 ,0 ,.,.) = 
 -0.6549 -0.6392 -0.6706  ...  -0.3882 -0.4510 -0.3020
 -0.6314 -0.5922 -0.5922  ...  -0.4667 -0.5529 -0.3647
 -0.5765 -0.5608 -0.5216  ...  -0.5529 -0.5137 -0.3412
           ...             ⋱             ...          
  0.4510  0.2627  0.1373  ...  -0.0510  0.0667 -0.0588
  0.4039  0.2627  0.1529  ...  -0.0118 -0.0667  0.0039
  0.2706  0.3020  0.1137  ...   0.0824 -0.0039  0.2471

(0 ,1 ,.,.) = 
 -0.3961 -0.3882 -0.4275  ...  -0.0196 -0.1294  0.0275
 -0.3804 -0.3176 -0.3490  ...  -0.0824 -0.2157 -0.0118
 -0.3098 -0.3098 -0.3020  ...  -0.2314 -0.1922  0.0275
           ...             ⋱             ...          
  0.3098  0.2784  0.1294  ...   0.1137  0.0980 -0.0431
  0.3412  0.2941  0.1216  ...   0.0353 -0.1451  0.0745
  0.2549  0.3098  0.0824  ...  -0.0588 -0.1294  0.3490

(0 ,2 ,.,.) = 
 -0.8118 -0.7882 -0.8510  ...  -0.2000 -0.3647 -0.2157
 -0.8118 -0.7804 -0.7961  ...  -0.3020 -0.4510 -0.3333
 -0.7020 -0.7725 -0.7333  ...  -0.4980 -0.4196 -0.2314
           ...   


labels: 

 8
 6
 4
 8
[torch.LongTensor of size 4]

inputs: 

(0 ,0 ,.,.) = 
  0.6784  0.6706  0.6784  ...   0.6471  0.6392  0.6627
  0.7098  0.7020  0.7098  ...   0.6706  0.6627  0.6784
  0.7020  0.6863  0.6941  ...   0.6627  0.6549  0.6706
           ...             ⋱             ...          
  0.3333  0.3255  0.3333  ...   0.1922  0.1922  0.1922
  0.3176  0.2784  0.2863  ...   0.1765  0.1765  0.1843
  0.3020  0.2941  0.3020  ...   0.2549  0.2392  0.2392

(0 ,1 ,.,.) = 
  0.7804  0.7725  0.7725  ...   0.7490  0.7412  0.7647
  0.8196  0.8039  0.8039  ...   0.7725  0.7647  0.7804
  0.8039  0.7882  0.7961  ...   0.7490  0.7412  0.7647
           ...             ⋱             ...          
  0.2784  0.2627  0.2627  ...   0.1686  0.1686  0.1765
  0.2784  0.2235  0.2078  ...   0.1608  0.1608  0.1608
  0.2627  0.2392  0.2235  ...   0.2314  0.2157  0.2078

(0 ,2 ,.,.) = 
  0.9294  0.9137  0.9216  ...   0.8824  0.8667  0.8980
  0.9529  0.9451  0.9451  ...   0.9059  0.8980  0.9137
  0.9373  

inputs: 

(0 ,0 ,.,.) = 
 -0.5216 -0.4902 -0.4824  ...  -0.5216 -0.4196 -0.4902
 -0.5373 -0.5529 -0.5686  ...  -0.2471 -0.2706 -0.4902
 -0.4431 -0.3804 -0.4039  ...   0.0039 -0.2392 -0.4980
           ...             ⋱             ...          
 -0.7647 -0.7569 -0.6706  ...  -0.3098 -0.1608 -0.0902
 -0.7490 -0.7490 -0.7176  ...  -0.6157 -0.4510 -0.1765
 -0.7333 -0.7647 -0.7725  ...  -0.7333 -0.6784 -0.3882

(0 ,1 ,.,.) = 
 -0.3490 -0.3412 -0.3412  ...  -0.4353 -0.3412 -0.4039
 -0.4275 -0.4196 -0.4353  ...  -0.2157 -0.2314 -0.4118
 -0.4196 -0.3176 -0.3255  ...  -0.0118 -0.2314 -0.4431
           ...             ⋱             ...          
 -0.7490 -0.7333 -0.6471  ...  -0.2706 -0.1137 -0.0431
 -0.7412 -0.7412 -0.7020  ...  -0.5686 -0.3961 -0.1216
 -0.7255 -0.7569 -0.7647  ...  -0.6706 -0.6235 -0.3255

(0 ,2 ,.,.) = 
 -0.3961 -0.4118 -0.4118  ...  -0.4588 -0.3490 -0.4275
 -0.4510 -0.4196 -0.4353  ...  -0.2235 -0.2235 -0.4353
 -0.4196 -0.2706 -0.2706  ...  -0.0118 -0.2157 -0.4431
        


(0 ,0 ,.,.) = 
  0.9843  0.8824  0.7882  ...   0.9686  0.9686  0.9922
  1.0000  0.7020  0.5843  ...   0.9922  1.0000  1.0000
  1.0000  0.5686  0.4902  ...   0.8902  0.9843  1.0000
           ...             ⋱             ...          
  1.0000  0.9922  0.9922  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  0.9843  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  0.9765  ...   1.0000  1.0000  1.0000

(0 ,1 ,.,.) = 
  0.9922  0.8588  0.6549  ...   0.9765  0.9765  1.0000
  0.9922  0.5686  0.3020  ...   0.9922  1.0000  1.0000
  0.9686  0.4118  0.1765  ...   0.8510  0.9529  0.9922
           ...             ⋱             ...          
  1.0000  0.9922  0.9922  ...   1.0000  1.0000  1.0000
  1.0000  0.9922  0.9843  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  0.9765  ...   1.0000  1.0000  1.0000

(0 ,2 ,.,.) = 
  0.9294  0.6549  0.3804  ...   0.9608  0.9686  0.9922
  0.9137  0.3333 -0.0353  ...   0.9451  1.0000  1.0000
  0.9373  0.2235 -0.1216  ...   0.6549  0.9137  1.0000
           ...   

...

inputs: 

(0 ,0 ,.,.) = 
 -0.5765 -0.3725 -0.4510  ...  -0.3961 -0.3647 -0.3961
 -0.5059 -0.4118 -0.3647  ...  -0.4353 -0.2392 -0.3490
 -0.4431 -0.2784 -0.2549  ...  -0.5059 -0.3412 -0.2627
           ...             ⋱             ...          
  0.1529 -0.0588 -0.2471  ...  -0.0196  0.0196  0.2706
  0.0039 -0.0431 -0.3020  ...  -0.1922  0.0039 -0.0902
 -0.1765 -0.1373 -0.3020  ...  -0.0745 -0.2000 -0.0902

(0 ,1 ,.,.) = 
 -0.4745 -0.2863 -0.3882  ...  -0.4275 -0.4275 -0.4196
 -0.4275 -0.3176 -0.2627  ...  -0.4588 -0.3020 -0.4039
 -0.3882 -0.1843 -0.1137  ...  -0.5294 -0.3961 -0.3255
           ...             ⋱             ...          
  0.1451 -0.1608 -0.3961  ...  -0.1608 -0.2941 -0.1294
  0.0275 -0.1529 -0.4431  ...  -0.2784 -0.2235 -0.3882
 -0.1059 -0.2235 -0.4118  ...  -0.1529 -0.3333 -0.2941

(0 ,2 ,.,.) = 
 -0.6235 -0.4510 -0.5608  ...  -0.4588 -0.5216 -0.5765
 -0.6157 -0.5294 -0.4980  ...  -0.5059 -0.4039 -0.5529
 -0.6000 -0.4353 -0.4039  ...  -0.5922 -0.5137 -0.4588
        

inputs: 

(0 ,0 ,.,.) = 
  0.9216  0.9451  0.9059  ...   0.8275  0.7020  0.5294
  0.9373  0.9529  0.9294  ...   0.8353  0.7098  0.5373
  0.9686  0.9765  0.9686  ...   0.8824  0.7569  0.6000
           ...             ⋱             ...          
  0.1216  0.0196 -0.6784  ...  -0.8745 -0.6941 -0.5843
  0.0510 -0.2784 -0.9059  ...  -0.7333 -0.7176 -0.7255
 -0.0745 -0.3255 -0.6706  ...  -0.6471 -0.6235 -0.5686

(0 ,1 ,.,.) = 
  0.9529  0.9765  0.9373  ...   0.8588  0.7255  0.5529
  0.9686  0.9843  0.9608  ...   0.8667  0.7333  0.5608
  0.9765  0.9922  0.9843  ...   0.8980  0.7804  0.6235
           ...             ⋱             ...          
  0.1922  0.0902 -0.6314  ...  -0.8510 -0.6392 -0.5294
  0.1294 -0.2157 -0.8667  ...  -0.6941 -0.6627 -0.6706
  0.0118 -0.2627 -0.6235  ...  -0.5843 -0.5608 -0.5137

(0 ,2 ,.,.) = 
  0.9765  0.9922  0.9608  ...   0.8902  0.7647  0.5922
  0.9922  1.0000  0.9765  ...   0.8902  0.7725  0.6000
  0.9922  1.0000  0.9922  ...   0.9216  0.8196  0.6627
        

Process Process-6:
Process Process-5:
Traceback (most recent call last):
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()


KeyboardInterrupt: 

  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
KeyboardInterrupt
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/riccardo/anaconda2/envs/py36/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(max

In [74]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

GroundTruth:    cat  ship  ship plane


In [75]:
outputs = net(Variable(images))

In [76]:
_, predicted = torch.max(outputs.data, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Predicted:    cat   car   car  ship


In [77]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 56 %


In [78]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
    
    
# These values are not anyway terrible, but i will try and make them better
# ill try just running it for 10 epochs first, and then see

Accuracy of plane : 51 %
Accuracy of   car : 70 %
Accuracy of  bird : 44 %
Accuracy of   cat : 44 %
Accuracy of  deer : 48 %
Accuracy of   dog : 48 %
Accuracy of  frog : 65 %
Accuracy of horse : 63 %
Accuracy of  ship : 61 %
Accuracy of truck : 69 %


In [79]:
net.cuda()

Net(
  (conv1): Conv2d (3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)

In [80]:
# will need to check why this is not working
inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

RuntimeError: Variable data has to be a tensor, but got Variable